In [1]:
from ast import increment_lineno
import numpy as np
import pandas as pd
import seaborn as sns 
import tensorflow as tf 

from tensorflow import keras as ks
from tensorflow.keras.layers import Dense
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

%matplotlib inline 

In [ ]:
boston_ds = datasets.load_boston()
columns = boston_ds.feature_names
target = boston_ds.target
data = pd.DataFrame(boston_ds.data, columns=columns).astype(np.float32)
data.head()

In [5]:
data['Price'] = target.astype(np.float32)
target = data['Price']
# data.head()

In [ ]:
data.isnull().sum()

In [ ]:
plt.figure(figsize=(11.7,8.27))
sns.displot(data['Price'], kde=True, height=7, aspect=2, bins=30)
plt.show()

In [ ]:
corr = data.corr().round(2)
plt.figure(figsize=(12.7,8.27))
sns.heatmap(data=corr, annot=True)

In [ ]:
features = ['RM', 'LSTAT']
plt.figure(figsize=(15.7,5.27))
for i, col in enumerate(features):
    plt.subplot(1, len(features) , i+1)
    x = data[col]
    y = target
    plt.scatter(x, y, marker='o')
    plt.title(col)
    plt.xlabel(col)
    plt.ylabel('Price')

In [6]:
X = pd.DataFrame(np.c_[data['LSTAT'], data['RM']], columns = ['LSTAT','RM'])
Y = data['Price']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=5)

In [8]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
mms.fit(X_train)
X_train = mms.transform(X_train)
X_test = mms.transform(X_test)

In [9]:
from tensorflow.keras.models import Sequential

model = Sequential(name='BostonHousingPrices')
model.add(Dense(50, input_dim=2, activation='relu' , name='Hidden-1'))
model.add(Dense(30, activation='relu' , name='Hidden-2'))
model.add(Dense(1, activation='relu' , name='Output'))


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mse', metrics=['mae'])

In [ ]:
hist = model.fit(X_train, y_train, batch_size=32, epochs=500, validation_split=0.2)

In [ ]:
figure = plt.gcf()
figure.set_size_inches((15, 5))
plt.title('Loss - Epoch Graphs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.plot(range(1, len(hist.history['loss']) + 1), hist.history['loss'])
plt.plot(range(1, len(hist.history['val_loss']) + 1), hist.history['val_loss'])
plt.legend(['Loss', 'Validation Loss'])
plt.show()

In [ ]:
figure = plt.gcf()
figure.set_size_inches((15, 5))
plt.title('Mean Absolute Error - Epoch Graph')
plt.xlabel('Epoch')
plt.ylabel('Mean Absolute Error')
plt.plot(range(1, len(hist.history['loss']) + 1), hist.history['loss'])
plt.plot(range(1, len(hist.history['val_loss']) + 1), hist.history['val_loss'])
plt.legend(['Mean Absolute Error', 'Validation Mean Absolute Error'])
plt.show()

In [14]:
result = model.evaluate(X_test, y_test)
for i in range(len(result)):
    print(f'{model.metrics_names[i]} ---> {result[i]}')

In [ ]:
prediction_data = np.array([6.0910, 10.46])
prediction_data = mms.transform(prediction_data.reshape(1, -1))
predict_result = model.predict(prediction_data)
print(f'Predicted price: {predict_result[0, 0]}')